<div class="alert alert-block alert-success">
<b>Imported libraries</b>
</div>

In [1]:
import chart_studio
import pandas as pd
# import yfinance as yf problem
import datetime as dt
import statsmodels.api as sm
import plotly.graph_objs as go
import chart_studio.plotly as py

from pandas_datareader import data

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

C:\Users\Sif\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
end = dt.datetime.now()
start = dt.date(end.year - 5, end.month - 1, end.day)
start_rf = dt.date(start.year, start.month + 1, start.day)

$$Capital\ Asset\ Pricing\ Model$$
$$R\ =\ R_f\ +\ \beta(R_m\ -\ R_f)$$

In [3]:
# For a brazilian stock as FLRY3 you have to put a .SA so it will be FLRY3.SA
r = data.DataReader(['AMZN'], 'yahoo', start, end)['Adj Close']
r = r.resample('M').last().pct_change()
r.head()

Symbols,AMZN
Date,
2016-03-31,NaN
2016-04-30,0.111094
2016-05-31,0.095817
2016-06-30,-0.009920
2016-07-31,0.060353


In [4]:
rm = data.DataReader(['^GSPC'], 'yahoo', start, end)['Adj Close']
rm = rm.resample('M').last().pct_change()
rm.head()

Symbols,^GSPC
Date,
2016-03-31,NaN
2016-04-30,0.002699
2016-05-31,0.015325
2016-06-30,0.000911
2016-07-31,0.035610


In [5]:
rf = data.DataReader('DGS1MO', 'fred', start_rf, end)
rf = rf.resample('M').last()
rf = rf/100
rf.head()

,DGS1MO
DATE,
2016-05-31,0.0027
2016-06-30,0.0020
2016-07-31,0.0019
2016-08-31,0.0026
2016-09-30,0.0020


In [6]:
# dropna to drop the first month
df = pd.DataFrame({'r': r['AMZN'],
                   'cuml': (1+r['AMZN']).cumprod(),
                   'rm': rm['^GSPC'], 
                   'rf': rf['DGS1MO'], 
                   'r - rf': r['AMZN'] - rf['DGS1MO'],
                   'rm - rf': rm['^GSPC'] - rf['DGS1MO']}).dropna()
df.head()

,r,cuml,rm,rf,r - rf,rm - rf
Date,,,,,,
2016-05-31,0.095817,1.217556,0.015325,0.0027,0.093117,0.012625
2016-06-30,-0.009920,1.205478,0.000911,0.0020,-0.011920,-0.001089
2016-07-31,0.060353,1.278233,0.035610,0.0019,0.058453,0.033710
2016-08-31,0.013640,1.295667,-0.001219,0.0026,0.011040,-0.003819
2016-09-30,0.088603,1.410468,-0.001234,0.0020,0.086603,-0.003234


In [7]:
#  An intercept is not included by default and should be added by the user
X1 = sm.add_constant(df['rm'])
model = sm.OLS(endog = df['r'], exog = X1, missing = 'drop')
results_capm = model.fit()
results_capm.params[0] = rf['DGS1MO'].tail(1)
results_capm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      r   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.334
Method:                 Least Squares   F-statistic:                     30.61
Date:                Fri, 30 Apr 2021   Prob (F-statistic):           7.93e-07
Time:                        00:03:03   Log-Likelihood:                 79.671
No. Observations:                  60   AIC:                            -155.3
Df Residuals:                      58   BIC:                            -151.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001      0.009      0.011      0.991      -0.017       0.018
rm             1.1470      0.196      5.858      0.000       0.755       1.539
==============================================================================
Omnibus:                        2.469   Durbin-Watson:                   1.725
Prob(Omnibus):                  0.291   Jarque-Bera (JB):                1.956
Skew:                           0.441   Prob(JB):                        0.376
Kurtosis:                       3.072   Cond. No.                         23.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
for i in r.columns:
    b = results_capm.params[1]
    print(i,'Beta:', b)

AMZN Beta: 1.1469602294386545


In [9]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df['rm'], y = df['r'], 
    name = 'Observed returns', 
    mode = 'markers',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(66, 133, 244)')))

fig.add_trace(go.Scatter(x = df['rm'], y = results_capm.fittedvalues,
    name = 'Required returns',
    mode = 'lines',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(234, 67, 53)')))

fig.add_annotation(text = r'$$R\ =\ R_f\ +\ \beta(R_m\ -\ R_f)$$',
                  xref = 'paper', yref = 'paper',
                  x = 0.05, y = 0.8, showarrow = False)

fig.update_layout(title_text = 'Capital Asset Pricing Model (CAPM)', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(
    x = 0, y = 1.5),
    xaxis_title = 'Monthly S&P 500 returns',
    yaxis_title = 'Monthly AMZN returns',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel = dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, 
         filename = 'Capital Asset Pricing Model (CAPM)', 
         auto_open = False)